In [194]:
import xarray as xr
import numpy as np
import pandas as pd
# import 

In [195]:
def select_season(data, season):
    if season == 'DJF':
        # December, January, February
        return data.sel(time=data.time.dt.month.isin([12, 1, 2]), method='nearest')
    elif season == 'MAM':
        # March, April, May
        return data.sel(time=data.time.dt.month.isin([3, 4, 5]), method='nearest')
    elif season == 'JJA':
        # June, July, August
        return data.sel(time=data.time.dt.month.isin([6, 7, 8]), method='nearest')
    elif season == 'SON':
        # September, October, November
        return data.sel(time=data.time.dt.month.isin([9, 10, 11]), method='nearest')
    elif season == 'DJFMAM':
        # December, January, February, March, April, May
        return data.sel(time=data.time.dt.month.isin([3, 4, 5, 12, 1, 2]), method='nearest')
    else:
        raise ValueError("Season must be one of 'DJF', 'MAM', 'JJA', 'DJFMAM' or 'SON'.")

def select_season_df(data, season):
    if season == 'DJF':
        # December, January, February
        return data.loc(data.index.time.month.isin([12, 1, 2]), method='nearest')
    elif season == 'MAM':
        # March, April, May
        return data.loc(data.index.time.month.isin([3, 4, 5]), method='nearest')
    elif season == 'JJA':
        # June, July, August
        return data.loc(data.index.time.month.isin([6, 7, 8]), method='nearest')
    elif season == 'SON':
        # September, October, November
        return data.loc(data.index.time.month.isin([9, 10, 11]), method='nearest')
    elif season == 'DJFMAM':
        # December, January, February, March, April, May
        return data.loc(data.index.time.month.isin([3, 4, 5, 12, 1, 2]), method='nearest')
    else:
        raise ValueError("Season must be one of 'DJF', 'MAM', 'JJA', 'DJFMAM' or 'SON'.")


In [196]:
# import xarray as xr
# import pandas as pd
# import numpy as np

# def extract_djf(data_array):
#     # Convert xarray DataArray time coordinate to pandas DatetimeIndex
#     time = pd.to_datetime(data_array.time.values)

#     # Create new time coordinates for the DJF season
#     new_times = []
#     for t in time:
#         print(t)
#         if t.month == 1 or t.month == 2:
#             new_times.append(pd.Timestamp(year=t.year - 1, month=t.month, day=t.day))

#     # Convert the list to a DataArray
#     new_time_da = xr.DataArray(new_times, dims='time', coords={'time': new_times})

#     # Update the time dimension in the DataArray
#     data_array.coords['time'] = new_time_da

#     return djf_data


In [197]:
folder = '/Users/fadhlilrizki/Documents/PhD/thesis/data/'
# fili   = 'agcd_v1_precip_calib_r005_daily_1981-2020.regrid.masked.nc'
fili   = 'agcd_v1_precip_calib_r025_daily_1978-2020.maskedshp.shift1day.nc'

In [198]:
ds     = xr.open_dataset(folder+fili)

In [199]:
precip  = ds.precip.sel(time=slice('1980-03-01','2020-05-30'))
th = 10.

In [200]:
precip

<xarray.DataArray 'precip' (time: 14701, lat: 132, lon: 163)>
[316306716 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1980-03-01T09:00:00 ... 2020-05-30T09:00:00
  * lon      (lon) float32 113.0 113.2 113.5 113.8 ... 152.8 153.0 153.2 153.5
  * lat      (lat) float32 -43.5 -43.25 -43.0 -42.75 ... -11.25 -11.0 -10.75
Attributes:
    standard_name:                 lwe_thickness_of_precipitation_amount
    long_name:                     Daily precipitation
    units:                         mm
    cell_methods:                  time: sum
    analysis_version_number:       3.01
    number_of_stations_reporting:  6820
    frequency:                     daily
    length_scale_for_analysis:     80.0
    coverage_content_type:         physicalMeasurement
    grid_mapping:                  spatial_ref

In [201]:
weights = np.cos(np.deg2rad(precip.lat))
weights.name = "weights"



In [202]:
precip_DJF = select_season(precip, 'DJF')
precip_MAM = select_season(precip, 'MAM')
precip_DJFMAM = select_season(precip, 'DJFMAM')


In [203]:

regions = {
            #kode  : [latS,latN,lonL,lonR]
            "TE"   : [-17, -11, 129, 137], 
            "CY"   : [-16, -10, 141, 146],
            "KY"   : [-20, -13.5, 120, 128],
            "QL"   : [-25, -20, 138, 145],
            "EC"   : [-23.5, -16, 146, 151],
            "NAU"  : [-25, -10, 120, 151]
          }

# regions = (/"TopEnd","CapeYork","Kimberley","Queensland","EastCoast"/)


In [204]:
regions['TE'][0]

-17

In [205]:
precip_DJF

<xarray.DataArray 'precip' (time: 3610, lat: 132, lon: 163)>
[77672760 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1980-12-01T09:00:00 ... 2020-02-29T09:00:00
  * lon      (lon) float32 113.0 113.2 113.5 113.8 ... 152.8 153.0 153.2 153.5
  * lat      (lat) float32 -43.5 -43.25 -43.0 -42.75 ... -11.25 -11.0 -10.75
Attributes:
    standard_name:                 lwe_thickness_of_precipitation_amount
    long_name:                     Daily precipitation
    units:                         mm
    cell_methods:                  time: sum
    analysis_version_number:       3.01
    number_of_stations_reporting:  6820
    frequency:                     daily
    length_scale_for_analysis:     80.0
    coverage_content_type:         physicalMeasurement
    grid_mapping:                  spatial_ref

In [206]:
region_code = regions.keys()
print(region_code)

monthly_DJF_precip = dict.fromkeys(region_code, 0)
monthly_DJFMAM_precip = dict.fromkeys(region_code, 0)
monthly_MAM_precip = dict.fromkeys(region_code, 0)



# precip_weighted = precip.weighted(weights)

# precip_
for rc in region_code:
    tmp = precip_DJF.sel(
                         lat=slice(regions[rc][0], regions[rc][1]),
                         lon=slice(regions[rc][2], regions[rc][3]),)
#     weights = np.cos(np.deg2rad(tmp.lat))
    monthly_DJF_precip[rc] = tmp
    # annual_DJFMAM_precip[rc] = precip_DJFMAM.sel(
    #                                         lat=slice(regions[rc][0], regions[rc][1]),
    #                                         lon=slice(regions[rc][2], regions[rc][3]),)
    # annual_MAM_precip[rc] = precip_MAM.sel(
    #                                         lat=slice(regions[rc][0], regions[rc][1]),
    #                                         lon=slice(regions[rc][2], regions[rc][3]),)
    

dict_keys(['TE', 'CY', 'KY', 'QL', 'EC', 'NAU'])


In [207]:
monthly_DJF_precip['TE']

<xarray.DataArray 'precip' (time: 3610, lat: 25, lon: 33)>
[2978250 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1980-12-01T09:00:00 ... 2020-02-29T09:00:00
  * lon      (lon) float32 129.0 129.2 129.5 129.8 ... 136.2 136.5 136.8 137.0
  * lat      (lat) float32 -17.0 -16.75 -16.5 -16.25 ... -11.5 -11.25 -11.0
Attributes:
    standard_name:                 lwe_thickness_of_precipitation_amount
    long_name:                     Daily precipitation
    units:                         mm
    cell_methods:                  time: sum
    analysis_version_number:       3.01
    number_of_stations_reporting:  6820
    frequency:                     daily
    length_scale_for_analysis:     80.0
    coverage_content_type:         physicalMeasurement
    grid_mapping:                  spatial_ref

In [208]:
weights

<xarray.DataArray 'weights' (lat: 132)>
array([0.7253744 , 0.72837096, 0.7313537 , 0.7343225 , 0.7372773 ,
       0.7402181 , 0.7431448 , 0.7460574 , 0.7489557 , 0.7518398 ,
       0.7547096 , 0.757565  , 0.76040596, 0.76323247, 0.76604444,
       0.7688418 , 0.7716246 , 0.77439266, 0.7771459 , 0.7798845 ,
       0.78260815, 0.78531694, 0.7880108 , 0.7906896 , 0.7933533 ,
       0.796002  , 0.79863554, 0.8012538 , 0.8038569 , 0.8064446 ,
       0.809017  , 0.811574  , 0.8141155 , 0.81664157, 0.81915206,
       0.8216469 , 0.8241262 , 0.82658976, 0.8290376 , 0.8314696 ,
       0.83388585, 0.8362862 , 0.83867055, 0.841039  , 0.8433914 ,
       0.84572786, 0.8480481 , 0.8503522 , 0.85264015, 0.85491186,
       0.8571673 , 0.8594064 , 0.8616291 , 0.8638355 , 0.8660254 ,
       0.8681988 , 0.87035567, 0.872496  , 0.8746197 , 0.87672675,
       0.87881714, 0.8808907 , 0.88294756, 0.88498765, 0.8870108 ,
       0.88901716, 0.8910065 , 0.89297897, 0.89493436, 0.89687276,
       0.89879405, 0.90069824, 0.90258527, 0.9044551 , 0.90630776,
       0.90814316, 0.9099613 , 0.91176206, 0.9135454 , 0.91531146,
       0.9170601 , 0.91879123, 0.92050487, 0.922201  , 0.9238795 ,
       0.9255405 , 0.92718387, 0.9288096 , 0.9304176 , 0.93200785,
       0.9335804 , 0.9351352 , 0.9366722 , 0.93819135, 0.9396926 ,
       0.941176  , 0.9426415 , 0.944089  , 0.94551855, 0.9469301 ,
       0.94832367, 0.9496991 , 0.95105654, 0.9523958 , 0.95371693,
       0.95501995, 0.9563048 , 0.9575714 , 0.95881975, 0.96004987,
       0.9612617 , 0.9624552 , 0.96363044, 0.9647873 , 0.9659258 ,
       0.96704596, 0.96814764, 0.9692309 , 0.9702957 , 0.9713421 ,
       0.9723699 , 0.97337925, 0.97437006, 0.97534233, 0.976296  ,
       0.9772311 , 0.9781476 , 0.97904545, 0.9799247 , 0.98078525,
       0.98162717, 0.9824504 ], dtype=float32)
Coordinates:
  * lat      (lat) float32 -43.5 -43.25 -43.0 -42.75 ... -11.25 -11.0 -10.75
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y

In [209]:
year = monthly_DJF_precip['TE'].resample(time='1Y').sum()
year

<xarray.DataArray 'precip' (time: 41, lat: 25, lon: 33)>
array([[[ 88.63965 ,  76.02637 ,  66.97266 , ...,  77.46875 ,
          65.20117 ,  49.817383],
        [ 86.49805 ,  73.47461 ,  68.96582 , ...,  97.13086 ,
          84.67285 ,  58.723633],
        [ 90.299805,  80.493164,  78.62891 , ..., 104.76367 ,
          91.58203 ,  66.14258 ],
        ...,
        [  0.      ,   0.      ,   0.      , ...,   0.      ,
           0.      ,   0.      ],
        [  0.      ,   0.      ,   0.      , ...,   0.      ,
           0.      ,   0.      ],
        [  0.      ,   0.      ,   0.      , ...,   0.      ,
           0.      ,   0.      ]],

       [[537.78613 , 544.1035  , 529.2256  , ..., 557.05566 ,
         570.33887 , 597.99414 ],
        [525.76953 , 537.8828  , 528.5625  , ..., 587.1992  ,
         603.8301  , 637.62695 ],
        [513.90625 , 548.7676  , 540.86035 , ..., 623.9199  ,
         677.8779  , 743.6504  ],
...
        [  0.      ,   0.      ,   0.      , ...,   0.      ,
           0.      ,   0.      ],
        [  0.      ,   0.      ,   0.      , ...,   0.      ,
           0.      ,   0.      ],
        [  0.      ,   0.      ,   0.      , ...,   0.      ,
           0.      ,   0.      ]],

       [[340.94434 , 338.16504 , 332.61426 , ..., 685.74316 ,
         736.3154  , 689.1416  ],
        [402.58594 , 384.6709  , 369.8916  , ..., 735.1377  ,
         765.76074 , 737.5049  ],
        [451.0537  , 434.78418 , 412.05566 , ..., 759.9951  ,
         759.6338  , 719.1455  ],
        ...,
        [  0.      ,   0.      ,   0.      , ...,   0.      ,
           0.      ,   0.      ],
        [  0.      ,   0.      ,   0.      , ...,   0.      ,
           0.      ,   0.      ],
        [  0.      ,   0.      ,   0.      , ...,   0.      ,
           0.      ,   0.      ]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1980-12-31 1981-12-31 ... 2020-12-31
  * lon      (lon) float32 129.0 129.2 129.5 129.8 ... 136.2 136.5 136.8 137.0
  * lat      (lat) float32 -17.0 -16.75 -16.5 -16.25 ... -11.5 -11.25 -11.0

In [210]:
DJF_wetdays = dict.fromkeys(region_code, 0)
annual_DJF_precip = dict.fromkeys(region_code, 0)
annual_DJF_wetdays = dict.fromkeys(region_code, 0)
annual_DJF_wetdays_region = dict.fromkeys(region_code, 0)
annual_DJF_precip_region = dict.fromkeys(region_code, 0)

# DJFMAM_wetdays = dict.fromkeys(region_code, 0)
# annual_DJFMAM_wetdays = dict.fromkeys(region_code, 0)
# annual_DJFMAM_wetdays_region = dict.fromkeys(region_code, 0)

# MAM_wetdays = dict.fromkeys(region_code, 0)
# annual_MAM_wetdays = dict.fromkeys(region_code, 0)
# annual_MAM_wetdays_region = dict.fromkeys(region_code, 0)

for rc in region_code:
    wd = monthly_DJF_precip[rc].copy()
    DJF_wetdays[rc]        = (wd > th).mean(("lon", "lat"))
    prc_mean = wd.mean(("lon","lat"))

    # print("wet")
    # print(DJF_wetdays[rc].head(31+31+28).sum())
    annual_DJF_wetdays[rc] = DJF_wetdays[rc].resample(time="Q-FEB").sum()
    annual_DJF_precip[rc] = prc_mean.resample(time="Q-FEB").sum()
    
    
    annual_DJF_precip_region[rc] = annual_DJF_precip[rc].dropna(dim='time')
    annual_DJF_wetdays_region[rc] = annual_DJF_wetdays[rc].dropna(dim='time')
    
    
    
    # print("sumQ")
    # print(annual_DJF_wetdays_region[rc].head())
    # DJFMAM_wetdays[rc]        = annual_DJFMAM_precip[rc].mean(("lon", "lat")) > th
    # annual_DJFMAM_wetdays[rc] = DJFMAM_wetdays[rc].resample(time="QS-MAR").sum()
    # annual_DJFMAM_wetdays_region[rc] = annual_DJFMAM_wetdays[rc].dropna(dim='time')
    
    # MAM_wetdays[rc]        = annual_MAM_precip[rc].mean(("lon", "lat")) > th
    # annual_MAM_wetdays[rc] = MAM_wetdays[rc].resample(time="QS-MAR").sum()
    # annual_MAM_wetdays_region[rc] = annual_MAM_wetdays[rc].dropna(dim='time')

In [211]:

annual_DJF_wetdays

{'TE': <xarray.DataArray 'precip' (time: 157)>
 array([17.37090909,         nan,         nan,         nan, 13.17212121,
                nan,         nan,         nan,  6.01090909,         nan,
                nan,         nan, 16.24484848,         nan,         nan,
                nan, 10.66909091,         nan,         nan,         nan,
        10.42787879,         nan,         nan,         nan, 16.6169697 ,
                nan,         nan,         nan, 10.50787879,         nan,
                nan,         nan, 10.59515152,         nan,         nan,
                nan,  8.40969697,         nan,         nan,         nan,
        20.38060606,         nan,         nan,         nan,  8.71030303,
                nan,         nan,         nan, 16.09454545,         nan,
                nan,         nan, 15.2       ,         nan,         nan,
                nan, 14.96727273,         nan,         nan,         nan,
        11.24848485,         nan,         nan,         nan, 20.06787879,
    

In [212]:
DJF_precip = dict.fromkeys(region_code, 0)
quarter_DJF_precip = dict.fromkeys(region_code, 0)
quarter_DJF_precip_region = dict.fromkeys(region_code, 0)

for rc in region_code:
    DJF_precip[rc]        = monthly_DJF_precip[rc].mean(("lat", "lon"))
    print("wet")
    
    print(DJF_precip[rc].head(31+31+28).sum())
    print(DJF_precip[rc])
    
    quarter_DJF_precip[rc] = DJF_precip[rc].resample(time="Q-FEB").sum()
   
    quarter_DJF_precip_region[rc] = quarter_DJF_precip[rc].dropna(dim='time')
    print(quarter_DJF_precip_region[rc])
    print("sumQ")
 

wet
<xarray.DataArray 'precip' ()>
array(751.25385, dtype=float32)
<xarray.DataArray 'precip' (time: 3610)>
array([ 2.1035345,  3.155754 ,  9.474434 , ..., 26.628979 ,  8.337741 ,
        4.710748 ], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1980-12-01T09:00:00 ... 2020-02-29T09:00:00
<xarray.DataArray 'precip' (time: 40)>
array([751.25385, 598.1221 , 296.223  , 697.6301 , 485.25647, 468.39877,
       761.2295 , 481.81854, 483.03336, 378.26807, 869.75665, 418.26886,
       733.7477 , 663.3477 , 695.0906 , 494.20624, 867.1745 , 709.0647 ,
       680.3465 , 672.94525, 923.2382 , 607.15314, 769.672  , 993.28   ,
       527.08923, 631.4221 , 580.68317, 852.3826 , 952.20087, 783.2164 ,
       986.7558 , 558.724  , 428.2    , 798.0219 , 684.29224, 623.4779 ,
       924.6679 , 762.15546, 424.2384 , 546.57446], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1981-02-28 1982-02-28 ... 2020-02-29
sumQ
wet
<xarray.DataArray 'precip' ()>
array(1139.5623, dtype=f

In [213]:
annual_DJF_precip_region
# annual_DJF_wetdays_region

{'TE': <xarray.DataArray 'precip' (time: 40)>
 array([751.25385, 598.1221 , 296.223  , 697.6301 , 485.25647, 468.39877,
        761.2295 , 481.81854, 483.03336, 378.26807, 869.75665, 418.26886,
        733.7477 , 663.3477 , 695.0906 , 494.20624, 867.1745 , 709.0647 ,
        680.3465 , 672.94525, 923.2382 , 607.15314, 769.672  , 993.28   ,
        527.08923, 631.4221 , 580.68317, 852.3826 , 952.20087, 783.2164 ,
        986.7558 , 558.724  , 428.2    , 798.0219 , 684.29224, 623.4779 ,
        924.6679 , 762.15546, 424.2384 , 546.57446], dtype=float32)
 Coordinates:
   * time     (time) datetime64[ns] 1981-02-28 1982-02-28 ... 2020-02-29,
 'CY': <xarray.DataArray 'precip' (time: 40)>
 array([1139.5623 ,  749.3561 ,  379.80243,  972.2298 ,  833.0673 ,
         742.492  ,  812.2798 ,  481.31134,  835.08923,  336.64423,
        1142.2952 ,  696.091  , 1323.2687 ,  759.63336,  618.65015,
         889.3228 ,  869.0888 , 1120.9928 , 1066.7871 ,  875.0239 ,
        1047.5026 ,  813.45245,  700

In [214]:
df_DJF_wd = pd.DataFrame(0, index=annual_DJF_wetdays_region['TE'].time, columns=region_code)
df_DJF_pr = pd.DataFrame(0, index=annual_DJF_precip_region['TE'].time, columns=region_code)

# df_DJFMAM_wd = pd.DataFrame(0, index=annual_DJFMAM_wetdays_region['TE'].time, columns=region_code)

# df_MAM_wd = pd.DataFrame(0, index=annual_MAM_wetdays_region['TE'].time, columns=region_code)

In [215]:
for rc in region_code:
    df_DJF_wd[rc] = annual_DJF_wetdays_region[rc]
    df_DJF_pr[rc] = annual_DJF_precip_region[rc]
    # df_DJFMAM_wd[rc] = annual_DJFMAM_wetdays_region[rc]
    # df_MAM_wd[rc] = annual_MAM_wetdays_region[rc]

In [216]:
#ADD SCATTERPLOT OF WAVE ACTIVITY(X) VS WET DAYS HERE-(Y)

In [217]:
df_DJF_pr.head()

,TE,CY,KY,QL,EC,NAU
1981-02-28,751.253845,1139.562256,596.451965,309.151581,568.861755,456.262115
1982-02-28,598.122070,749.356079,713.819458,114.392342,219.686234,340.282990
1983-02-28,296.222992,379.802429,290.477936,53.784164,213.201508,165.209885
1984-02-29,697.630127,972.229797,538.506104,280.563324,313.359833,382.769409
1985-02-28,485.256470,833.067322,321.766083,117.151634,253.338806,232.308609


In [218]:
df_DJF_pr.head()

,TE,CY,KY,QL,EC,NAU
1981-02-28,751.253845,1139.562256,596.451965,309.151581,568.861755,456.262115
1982-02-28,598.122070,749.356079,713.819458,114.392342,219.686234,340.282990
1983-02-28,296.222992,379.802429,290.477936,53.784164,213.201508,165.209885
1984-02-29,697.630127,972.229797,538.506104,280.563324,313.359833,382.769409
1985-02-28,485.256470,833.067322,321.766083,117.151634,253.338806,232.308609


In [219]:
# sorted_df_TE_DJF = df_DJF_wd['TE'].dropna().sort_values(ascending=False)
# sorted_df_CY_DJF = df_DJF_wd['CY'].dropna().sort_values(ascending=False)
# sorted_df_KY_DJF = df_DJF_wd['KY'].dropna().sort_values(ascending=False)
# sorted_df_QL_DJF = df_DJF_wd['QL'].dropna().sort_values(ascending=False)
# sorted_df_EC_DJF = df_DJF_wd['EC'].dropna().sort_values(ascending=False)
# sorted_df_NAU_DJF = df_DJF_wd['NAU'].dropna().sort_values(ascending=False)

sorted_df_TE_DJF = df_DJF_pr['TE'].dropna().sort_values(ascending=False)
sorted_df_CY_DJF = df_DJF_pr['CY'].dropna().sort_values(ascending=False)
sorted_df_KY_DJF = df_DJF_pr['KY'].dropna().sort_values(ascending=False)
sorted_df_QL_DJF = df_DJF_pr['QL'].dropna().sort_values(ascending=False)
sorted_df_EC_DJF = df_DJF_pr['EC'].dropna().sort_values(ascending=False)
sorted_df_NAU_DJF = df_DJF_pr['NAU'].dropna().sort_values(ascending=False)

# sorted_df_TE_DJFMAM = df_DJFMAM_wd['TE'].sort_values(ascending=False)
# sorted_df_CY_DJFMAM = df_DJFMAM_wd['CY'].sort_values(ascending=False)
# sorted_df_KY_DJFMAM = df_DJFMAM_wd['KY'].sort_values(ascending=False)
# sorted_df_QL_DJFMAM = df_DJFMAM_wd['QL'].sort_values(ascending=False)
# sorted_df_EC_DJFMAM = df_DJFMAM_wd['EC'].sort_values(ascending=False)
# sorted_df_NAU_DJFMAM = df_DJFMAM_wd['NAU'].sort_values(ascending=False)

# sorted_df_TE_MAM = df_MAM_wd['TE'].dropna().sort_values(ascending=False)
# sorted_df_CY_MAM = df_MAM_wd['CY'].dropna().sort_values(ascending=False)
# sorted_df_KY_MAM = df_MAM_wd['KY'].dropna().sort_values(ascending=False)
# sorted_df_QL_MAM = df_MAM_wd['QL'].dropna().sort_values(ascending=False)
# sorted_df_EC_MAM = df_MAM_wd['EC'].dropna().sort_values(ascending=False)
# sorted_df_NAU_MAM = df_MAM_wd['NAU'].dropna().sort_values(ascending=False)

In [220]:
std_wd_TE = df_DJF_wd['TE'].std()
avg_wd_TE = df_DJF_wd['TE'].mean()

std_pr_TE = df_DJF_pr['TE'].std()
avg_pr_TE = df_DJF_pr['TE'].mean()

print(std_pr_TE)
print(avg_pr_TE)

178.73426818847656
664.0657958984375


In [221]:
sorted_df_TE_DJF

2004-02-29    993.280029
2011-02-28    986.755798
2009-02-28    952.200867
2017-02-28    924.667908
2001-02-28    923.238220
1991-02-28    869.756653
1997-02-28    867.174500
2008-02-29    852.382629
2014-02-28    798.021912
2010-02-28    783.216370
2003-02-28    769.671997
2018-02-28    762.155457
1987-02-28    761.229492
1981-02-28    751.253845
1993-02-28    733.747681
1998-02-28    709.064697
1984-02-29    697.630127
1995-02-28    695.090576
2015-02-28    684.292236
1999-02-28    680.346497
2000-02-29    672.945251
1994-02-28    663.347717
2006-02-28    631.422119
2016-02-29    623.477905
2002-02-28    607.153137
1982-02-28    598.122070
2007-02-28    580.683167
2012-02-29    558.723999
2020-02-29    546.574463
2005-02-28    527.089233
1996-02-29    494.206238
1985-02-28    485.256470
1989-02-28    483.033356
1988-02-29    481.818542
1986-02-28    468.398773
2013-02-28    428.200012
2019-02-28    424.238403
1992-02-29    418.268860
1990-02-28    378.268066
1983-02-28    296.222992


In [222]:
# Calculate mean and standard deviation
mean_value = sorted_df_TE_DJF.mean()
std_value = sorted_df_TE_DJF.std()

# Calculate thresholds
upper_threshold = mean_value + 0.5 * std_value
lower_threshold = mean_value - 0.5 * std_value

# Filter the DataFrame
wettest_DJF = sorted_df_TE_DJF[sorted_df_TE_DJF >= upper_threshold]
mid_DJF = sorted_df_TE_DJF[(sorted_df_TE_DJF <= upper_threshold) & (sorted_df_TE_DJF >= lower_threshold)]
driest_DJF = sorted_df_TE_DJF[sorted_df_TE_DJF <= lower_threshold]

In [223]:
folder = '/Users/fadhlilrizki/Documents/PhD/thesis/dataout/STSA/year_choose/'


pd.DataFrame(wettest_DJF.index.year).to_csv(folder + 'totalrain_wettest_TE_DJF.csv',index=False, header=False)
pd.DataFrame(driest_DJF.index.year).to_csv(folder + 'totalrain_driest_TE_DJF.csv',index=False, header=False)
pd.DataFrame(mid_DJF.index.year).to_csv(folder + 'totalrain_mid_TE_DJF.csv',index=False, header=False)


In [224]:
wettest_DJF

2004-02-29    993.280029
2011-02-28    986.755798
2009-02-28    952.200867
2017-02-28    924.667908
2001-02-28    923.238220
1991-02-28    869.756653
1997-02-28    867.174500
2008-02-29    852.382629
2014-02-28    798.021912
2010-02-28    783.216370
2003-02-28    769.671997
2018-02-28    762.155457
1987-02-28    761.229492
Name: TE, dtype: float32

In [225]:
seas='DJF'

oni = pd.read_csv('ONI.csv', sep=',')

oni_use = oni[(oni['YR']>=1980) & (oni['YR']<=2020)]

# seasons_to_filter = ['NDJ', 'DJF', 'FMA', 'MAM','AMJ']

if seas=='DJF':
    seasons_to_filter = ['NDJ', 'DJF','JFM']
elif seas=='MAM':
    seasons_to_filter = ['FMA', 'MAM', 'AMJ',]
oni_use = oni_use[oni_use['SEAS'].isin(seasons_to_filter)]


In [226]:
oni_use_sum = oni_use.copy()

In [227]:
event_counts = oni_use_sum.groupby('YR')['EVENT'].value_counts().unstack(fill_value=0)
event_counts.columns = ['La-Nina', 'Neutral', 'El-Nino']

In [228]:
event_counts['YR'] = event_counts.index

In [229]:
event_counts.index = np.arange(0,len(event_counts))

In [230]:
event_counts

,La-Nina,Neutral,El-Nino,YR
0,0,2,1,1980
1,0,3,0,1981
2,0,2,1,1982
3,1,0,2,1983
4,2,1,0,1984
5,2,1,0,1985
6,0,2,1,1986
7,0,0,3,1987
8,1,0,2,1988
9,2,1,0,1989


In [231]:
oni_use_sum.index = oni_use_sum['YR']

In [232]:
oni_use_sum

,SEAS,YR,TOTAL,ANOM,EVENT
YR,,,,,
1980,DJF,1980,27.06,0.59,1
1980,JFM,1980,27.13,0.46,0
1980,NDJ,1980,26.48,-0.01,0
1981,DJF,1981,26.32,-0.26,0
1981,JFM,1981,26.32,-0.50,0
...,...,...,...,...,...
2019,JFM,2019,27.59,0.72,1
2019,NDJ,2019,27.17,0.55,1
2020,DJF,2020,27.14,0.50,0


In [233]:
oni_djf = oni_use[oni_use['SEAS']=='DJF']

In [234]:
df_wettest_DJF = pd.DataFrame(wettest_DJF)
df_driest_DJF = pd.DataFrame(driest_DJF)
df_mid_DJF = pd.DataFrame(mid_DJF)

In [235]:
df_wettest_DJF['DATE'] = df_wettest_DJF.index
df_wettest_DJF['YEAR'] = df_wettest_DJF['DATE'].dt.year

df_driest_DJF['DATE'] = df_driest_DJF.index
df_driest_DJF['YEAR'] = df_driest_DJF['DATE'].dt.year

df_mid_DJF['DATE'] = df_mid_DJF.index
df_mid_DJF['YEAR'] = df_mid_DJF['DATE'].dt.year

In [236]:
df_wettest_enso = df_wettest_DJF.merge(oni_djf[['YR', 'SEAS','EVENT']], left_on='YEAR', right_on='YR', how='left')
df_driest_enso = df_driest_DJF.merge(oni_djf[['YR', 'SEAS','EVENT']], left_on='YEAR', right_on='YR', how='left')
df_mid_enso = df_mid_DJF.merge(oni_djf[['YR', 'SEAS','EVENT']], left_on='YEAR', right_on='YR', how='left')

In [237]:
df_wettest_enso_allseas = df_wettest_DJF.merge(event_counts[['YR', 'La-Nina', 'Neutral', 'El-Nino']], left_on='YEAR', right_on='YR', how='left')
df_driest_enso_allseas  = df_driest_DJF.merge(event_counts[['YR', 'La-Nina', 'Neutral', 'El-Nino']], left_on='YEAR', right_on='YR', how='left')
df_mid_enso_allseas     = df_mid_DJF.merge(event_counts[['YR', 'La-Nina', 'Neutral', 'El-Nino']], left_on='YEAR', right_on='YR', how='left')



In [238]:
df_driest_enso_allseas.sum(axis=0)

/var/folders/41/txq0cr4j1kl0vqjl56x7l60w0000gp/T/ipykernel_44774/2486865646.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_driest_enso_allseas.sum(axis=0)


TE          5990.299805
YEAR       25978.000000
YR         25978.000000
La-Nina       12.000000
Neutral       15.000000
El-Nino       12.000000
dtype: float64

In [239]:
df_wettest_enso_allseas.sum(axis=0)

/var/folders/41/txq0cr4j1kl0vqjl56x7l60w0000gp/T/ipykernel_44774/300419692.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_wettest_enso_allseas.sum(axis=0)


TE         11243.751953
YEAR       26070.000000
YR         26070.000000
La-Nina       14.000000
Neutral       12.000000
El-Nino       13.000000
dtype: float64

In [240]:
df_mid_enso_allseas.sum()

/var/folders/41/txq0cr4j1kl0vqjl56x7l60w0000gp/T/ipykernel_44774/2824772872.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mid_enso_allseas.sum()


TE          9328.578125
YEAR       27972.000000
YR         27972.000000
La-Nina       14.000000
Neutral       15.000000
El-Nino       13.000000
dtype: float64

In [241]:
def classify_enso(event):
    if event == 1:
        return 'El-Nino'
    elif event == -1:
        return 'La-Nina'  # or any other condition you want to add
    else:
        return 'Neutral'  # default case for other values

In [242]:
def classify_enso(event):
    if event == 1:
        return 'El-Nino'
    elif event == -1:
        return 'La-Nina'  # or any other condition you want to add
    else:
        return 'Neutral'  # default case for other values

In [243]:
local_sst = pd.read_csv('North_AU_evap_sharmila.csv',header=None
                       )

In [244]:
local_mean = local_sst[1].mean()
local_std  = local_sst[1].std()

print(local_mean)
print(local_std)

0.032682926829268294
0.22911813972538164


In [245]:
def classify_local(data,key,mean,std):
    
    val = []
    for idata in data[key]:
        if idata > mean+0.5*std:
            val.append('less evaporation')
        elif idata < mean-0.5*std:
            val.append('more evaporation')  # or any other condition you want to add
        else:
            val.append('Neutral')  # default case for other values
            
    return val

In [246]:
local_sst['event'] = classify_local(local_sst,1,local_mean,local_std)

In [247]:
local_sst = local_sst.rename(columns={0:'YEAR',1:'EVAP_local'})

In [248]:
local_sst

,YEAR,EVAP_local,event
0,1981,-0.34,more evaporation
1,1982,-0.14,more evaporation
2,1983,0.23,less evaporation
3,1984,0.27,less evaporation
4,1985,0.09,Neutral
5,1986,0.31,less evaporation
6,1987,-0.14,more evaporation
7,1988,0.32,less evaporation
8,1989,0.05,Neutral
9,1990,0.20,less evaporation


In [249]:
df_wettest_enso['ENSO'] = df_wettest_enso['EVENT'].apply(classify_enso)
df_driest_enso['ENSO'] = df_driest_enso['EVENT'].apply(classify_enso)
df_mid_enso['ENSO'] = df_mid_enso['EVENT'].apply(classify_enso)

In [250]:
df_wettest_enso

,TE,DATE,YEAR,YR,SEAS,EVENT,ENSO
0,993.280029,2004-02-29,2004,2004,DJF,0,Neutral
1,986.755798,2011-02-28,2011,2011,DJF,-1,La-Nina
2,952.200867,2009-02-28,2009,2009,DJF,-1,La-Nina
3,924.667908,2017-02-28,2017,2017,DJF,0,Neutral
4,923.238220,2001-02-28,2001,2001,DJF,-1,La-Nina
5,869.756653,1991-02-28,1991,1991,DJF,0,Neutral
6,867.174500,1997-02-28,1997,1997,DJF,0,Neutral
7,852.382629,2008-02-29,2008,2008,DJF,-1,La-Nina
8,798.021912,2014-02-28,2014,2014,DJF,0,Neutral
9,783.216370,2010-02-28,2010,2010,DJF,1,El-Nino


In [251]:
df_wettest_enso = df_wettest_enso.merge(local_sst[['YEAR','EVAP_local', 'event']], left_on='YR', right_on='YEAR', how='left')
df_mid_enso = df_mid_enso.merge(local_sst[['YEAR','EVAP_local', 'event']], left_on='YR', right_on='YEAR', how='left')
df_driest_enso = df_driest_enso.merge(local_sst[['YEAR','EVAP_local', 'event']], left_on='YR', right_on='YEAR', how='left')

In [252]:
df_wettest_enso

,TE,DATE,YEAR_x,YR,SEAS,EVENT,ENSO,YEAR_y,EVAP_local,event
0,993.280029,2004-02-29,2004,2004,DJF,0,Neutral,2004,-0.20,more evaporation
1,986.755798,2011-02-28,2011,2011,DJF,-1,La-Nina,2011,-0.32,more evaporation
2,952.200867,2009-02-28,2009,2009,DJF,-1,La-Nina,2009,-0.25,more evaporation
3,924.667908,2017-02-28,2017,2017,DJF,0,Neutral,2017,-0.28,more evaporation
4,923.238220,2001-02-28,2001,2001,DJF,-1,La-Nina,2001,-0.00,Neutral
5,869.756653,1991-02-28,1991,1991,DJF,0,Neutral,1991,-0.24,more evaporation
6,867.174500,1997-02-28,1997,1997,DJF,0,Neutral,1997,-0.17,more evaporation
7,852.382629,2008-02-29,2008,2008,DJF,-1,La-Nina,2008,-0.29,more evaporation
8,798.021912,2014-02-28,2014,2014,DJF,0,Neutral,2014,0.08,Neutral
9,783.216370,2010-02-28,2010,2010,DJF,1,El-Nino,2010,-0.02,Neutral


In [253]:
print(df_wettest_enso)
print(df_driest_enso)
print(df_mid_enso)

            TE       DATE  YEAR_x    YR SEAS  EVENT     ENSO  YEAR_y  \
0   993.280029 2004-02-29    2004  2004  DJF      0  Neutral    2004   
1   986.755798 2011-02-28    2011  2011  DJF     -1  La-Nina    2011   
2   952.200867 2009-02-28    2009  2009  DJF     -1  La-Nina    2009   
3   924.667908 2017-02-28    2017  2017  DJF      0  Neutral    2017   
4   923.238220 2001-02-28    2001  2001  DJF     -1  La-Nina    2001   
5   869.756653 1991-02-28    1991  1991  DJF      0  Neutral    1991   
6   867.174500 1997-02-28    1997  1997  DJF      0  Neutral    1997   
7   852.382629 2008-02-29    2008  2008  DJF     -1  La-Nina    2008   
8   798.021912 2014-02-28    2014  2014  DJF      0  Neutral    2014   
9   783.216370 2010-02-28    2010  2010  DJF      1  El-Nino    2010   
10  769.671997 2003-02-28    2003  2003  DJF      1  El-Nino    2003   
11  762.155457 2018-02-28    2018  2018  DJF     -1  La-Nina    2018   
12  761.229492 1987-02-28    1987  1987  DJF      1  El-Nino    

In [254]:
df_wettest_enso

,TE,DATE,YEAR_x,YR,SEAS,EVENT,ENSO,YEAR_y,EVAP_local,event
0,993.280029,2004-02-29,2004,2004,DJF,0,Neutral,2004,-0.20,more evaporation
1,986.755798,2011-02-28,2011,2011,DJF,-1,La-Nina,2011,-0.32,more evaporation
2,952.200867,2009-02-28,2009,2009,DJF,-1,La-Nina,2009,-0.25,more evaporation
3,924.667908,2017-02-28,2017,2017,DJF,0,Neutral,2017,-0.28,more evaporation
4,923.238220,2001-02-28,2001,2001,DJF,-1,La-Nina,2001,-0.00,Neutral
5,869.756653,1991-02-28,1991,1991,DJF,0,Neutral,1991,-0.24,more evaporation
6,867.174500,1997-02-28,1997,1997,DJF,0,Neutral,1997,-0.17,more evaporation
7,852.382629,2008-02-29,2008,2008,DJF,-1,La-Nina,2008,-0.29,more evaporation
8,798.021912,2014-02-28,2014,2014,DJF,0,Neutral,2014,0.08,Neutral
9,783.216370,2010-02-28,2010,2010,DJF,1,El-Nino,2010,-0.02,Neutral


In [255]:
df_driest_enso

,TE,DATE,YEAR_x,YR,SEAS,EVENT,ENSO,YEAR_y,EVAP_local,event
0,558.723999,2012-02-29,2012,2012,DJF,-1,La-Nina,2012,0.30,less evaporation
1,546.574463,2020-02-29,2020,2020,DJF,0,Neutral,2020,0.23,less evaporation
2,527.089233,2005-02-28,2005,2005,DJF,1,El-Nino,2005,0.13,Neutral
3,494.206238,1996-02-29,1996,1996,DJF,-1,La-Nina,1996,0.21,less evaporation
4,485.256470,1985-02-28,1985,1985,DJF,-1,La-Nina,1985,0.09,Neutral
5,483.033356,1989-02-28,1989,1989,DJF,-1,La-Nina,1989,0.05,Neutral
6,481.818542,1988-02-29,1988,1988,DJF,1,El-Nino,1988,0.32,less evaporation
7,468.398773,1986-02-28,1986,1986,DJF,0,Neutral,1986,0.31,less evaporation
8,428.200012,2013-02-28,2013,2013,DJF,0,Neutral,2013,0.36,less evaporation
9,424.238403,2019-02-28,2019,2019,DJF,1,El-Nino,2019,0.21,less evaporation


In [256]:
df_mid_enso

,TE,DATE,YEAR_x,YR,SEAS,EVENT,ENSO,YEAR_y,EVAP_local,event
0,751.253845,1981-02-28,1981,1981,DJF,0,Neutral,1981,-0.34,more evaporation
1,733.747681,1993-02-28,1993,1993,DJF,0,Neutral,1993,-0.22,more evaporation
2,709.064697,1998-02-28,1998,1998,DJF,1,El-Nino,1998,0.30,less evaporation
3,697.630127,1984-02-29,1984,1984,DJF,-1,La-Nina,1984,0.27,less evaporation
4,695.090576,1995-02-28,1995,1995,DJF,1,El-Nino,1995,0.23,less evaporation
5,684.292236,2015-02-28,2015,2015,DJF,1,El-Nino,2015,0.16,less evaporation
6,680.346497,1999-02-28,1999,1999,DJF,-1,La-Nina,1999,-0.31,more evaporation
7,672.945251,2000-02-29,2000,2000,DJF,-1,La-Nina,2000,0.05,Neutral
8,663.347717,1994-02-28,1994,1994,DJF,0,Neutral,1994,0.31,less evaporation
9,631.422119,2006-02-28,2006,2006,DJF,-1,La-Nina,2006,-0.20,more evaporation


In [257]:
df_wettest_enso.to_csv(folder + 'details_totalrain_wettest_TE_DJF.csv')
df_driest_enso.to_csv(folder + 'details_totalrain_driest_TE_DJF.csv')
df_mid_enso.to_csv(folder + 'details_totalrain_moderate_TE_DJF.csv')
